Link: https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/reproject-raster/

In [ ]:
import os
import numpy as np
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import earthpy as et

# Get data and set working directory
et.data.get_data("colorado-flood")
os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

In [ ]:
# Define relative path to file
lidar_dem_path = os.path.join("data", "colorado-flood", "spatial", 
                              "boulder-leehill-rd", "pre-flood", "lidar",
                              "pre_DTM.tif")

lidar_dem = rio.open(lidar_dem_path)
print(lidar_dem.meta)

In [ ]:
dst_crs = 'EPSG:3857' # CRS for web meractor 

projected_lidar_dem_path = os.path.join("data", "colorado-flood", "spatial", 
                                        "boulder-leehill-rd", "pre-flood", "lidar",
                                        "pre_DTM_reproject.tif")

with rio.open(lidar_dem_path) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rio.open(projected_lidar_dem_path, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rio.band(src, i),
                destination=rio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

In [ ]:
lidar_dem2 = rio.open(projected_lidar_dem_path)

print(lidar_dem2.meta)

In [ ]:
lidar_dem2.close()

In [ ]:
def reproject_et(inpath, outpath, new_crs):
    dst_crs = new_crs # CRS for web meractor 

    with rio.open(inpath) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rio.open(outpath, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rio.band(src, i),
                    destination=rio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

In [ ]:
reproject_et(inpath = os.path.join("data", "colorado-flood", "spatial", 
                                   "boulder-leehill-rd", "pre-flood", "lidar", "pre_DTM.tif"), 
             outpath = os.path.join("data", "colorado-flood", "spatial", 
                                    "boulder-leehill-rd", "pre-flood", "lidar", "pre_DTM_reproject_2.tif"), 
             new_crs = 'EPSG:3857')

In [ ]:
projected_lidar_dem_path = os.path.join("data", "colorado-flood", "spatial", 
                                        "boulder-leehill-rd", "pre-flood", "lidar",
                                        "pre_DTM_reproject_2.tif")

# Check to make sure function worked, then close raster
lidar_dem3 = rio.open(projected_lidar_dem_path)
print(lidar_dem3.meta)

lidar_dem3.close()